# Stock Analysis of Recession Resilient Companies

## Introduction

#### The goal for this code is to find stocks that have been resilient to past major US recessions, analyze what they have in common and use this information to pick which ones are most likely to keep on staying afloat during economic downturns.

In [19]:
from pandas_datareader import data
import pandas as pd
import matplotlib.pyplot as plt

In [20]:
#Function to provide financial data when called
def readmydata(tickers, start_date, end_date):
    financial_data = data.DataReader(tickers, 'yahoo', start_date, end_date)
    df = pd.DataFrame(financial_data)
    df.columns = df.columns
    output = df.stack(level=-1)
    return output

In [21]:
#Function takes as input dataframe and picks columns
def pickmydata(data_pack, attributes, num_days):
    col = data_pack[attributes]
    col.head()
    mytickers_name = data_pack.index.get_level_values('Symbols')
    mytickers_name = mytickers_name.unique()
    for x in mytickers_name:      
        #Query dataframe for the columns user asked for
        data_ticker = data_pack[data_pack.index.get_level_values('Symbols') == x]
        #Object series indexed by date
        data_ticker_attributes = col.loc[:, x]
        #20-day moving average
        short_rolling_tickers = data_ticker_attributes.rolling(window = num_days).mean()
        #Plot num_days referring to how many days are in a unit of rolling average
        fig, ax = plt.subplots(figsize=(9,5))
        ax.plot(data_ticker_attributes.index, data_ticker_attributes, label=x)
        ax.plot(short_rolling_tickers.index, short_rolling_tickers, label= 'Rolling average')
        ax.set_xlabel('Date')
        #Find only requested ticker
        ax.set_ylabel(attributes + ' Price ($)')
        ax.legend()
        output = data_ticker_attributes.describe()
    return output

In [4]:
#Function taking as input companies, date range and output stock returns
#Create new dataframe
#Calculate daily returns of each stock (Difference/og Price or Log(D/OG))
#Companies are in the header row
def stockreturns(data_pack):
    #Creating dataframe with dates as a column and company as header
    #Calculate daily stock returns by comparing previous day's closing price with current closing price
    #Use [f(x+1)-f(x)]/f(x) for relative returns
    ##close_price = data_pack.loc[data_pack['Attributes'] == 'Close']
    #Find a way to go back a date
    #Tried creating new dataset using returns_df = data_pack['Date', 'Symbols']
    #new_dataset = data_pack.drop('Adj Close', inplace=True, axis=1)
    #Using subsets not working
    #Next, try dropping attributes column and 'add' returns columns to calculate returns from old dataframe
    returns_df = datapack[['Date','Symbols']].copy()
    output = returns_df
    return output

## Resilient Companies

#### Companies that survive recessions tend to have basic necessities needed by consumers. This is because consumer staples and food are always in demand, even during economic downturns. Examples of such companies the likes of Walmart, Kroger and General Mills

In [22]:
data_pack = readmydata(tickers=['^GSPC', 'WMT', 'KR', 'GIS', 'HRL', 'CPB'], start_date='2007-12-01', end_date='2009-06-30')
data_pack.head(10)
#pickmydata(data_pack, "Close", 30)

Attributes            Adj Close        Close         High          Low  \
Date       Symbols                                                       
2007-12-03 ^GSPC    1472.420044  1472.420044  1481.160034  1470.079956   
           WMT        34.368843    47.869999    48.310001    47.570000   
           KR         11.215734    14.445000    14.675000    14.250000   
           GIS        18.718275    29.985001    30.235001    29.855000   
           HRL         7.479504     9.907500     9.970000     9.820000   
           CPB        23.646526    36.669998    36.779999    36.430000   
2007-12-04 ^GSPC    1462.790039  1462.790039  1471.339966  1460.660034   
           WMT        34.598602    48.189999    48.560001    47.490002   
           KR         10.982803    14.145000    14.530000    14.145000   
           GIS        18.696430    29.950001    30.150000    29.745001   

Attributes                 Open        Volume  
Date       Symbols                             
2007-12-03 ^GSPC    1479.630005  3.323250e+09  
           WMT        47.709999  1.227610e+07  
           KR         14.455000  1.024120e+07  
           GIS        29.855000  4.256000e+06  
           HRL         9.877500  1.088800e+06  
           CPB        36.549999  1.846900e+06  
2007-12-04 ^GSPC    1471.339966  3.343620e+09  
           WMT        47.490002  1.646050e+07  
           KR         14.330000  1.090900e+07  
           GIS        29.745001  4.290400e+06

#### Looking for trends, we can see that retail company stocks would hit a dip during the recession, but the precentage of the dip would be less extreme than that of the S&P.

## Sensitive Companies

#### Companies that tend to perform terribly are those who depend on consumers being able to afford big ticket item, such as cars, housing and plane tickets. The reasons depend on the industry, for example, though the car industry is quickly  becoming a necessity, are quite expensive and that alone makes the demand for them fall during economic downturns.

In [15]:
data_pack = readmydata(tickers=['^GSPC', 'AMT', 'F', 'BA', 'HST', 'AAL'], start_date='2007-12-01', end_date='2009-06-30')
#data_pack.head(10)
pickmydata(data_pack, "Close", 30)

KeyError: 'Requested level (Symbols) does not match index name (Date)'

#### These companies have got a dip that is either as sharp or worse than the S&P 500. Unlike the discount retail companies, the industries that these commpanies belong to rely on consumer spending large amounts of money, a feature that is stressed during economic downturns.

In [24]:
returns_df = data_pack[['Date','Symbols']].copy()
output = returns_df

KeyError: "None of [Index(['Date', 'Symbols'], dtype='object', name='Attributes')] are in the [columns]"